In [1]:
## The following code ensures that all functions and init files are reloaded before executions.
%load_ext autoreload
%autoreload 2

In [2]:
import geopandas as gpd
from insitupy.utils.geo import parse_geopandas
from insitupy.utils.io import read_json
import numpy as np
import shapely
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from insitupy import XeniumData
from rasterio.features import rasterize
from pathlib import Path

In [3]:
baysor_folder = Path(r"C:\Users\ge37voy\OneDrive - TUM\data\2315_VascularCancer_Mogler\2315-02-glioblastoma\20240112__122324__2315-02-Glioblastoma\baysor\baysor_output-0017313__N-42-20-T__20240219__155112")
#annotfile = Path(r"C:\Users\ge37voy\Github\InSituPy\notebooks\demo_annotations\annotations-slide_id__sample_id__demo2.geojson")
xdfile = Path(r"C:\Users\ge37voy\OneDrive - TUM\data\2315_VascularCancer_Mogler\2315-02-glioblastoma\20240112__122324__2315-02-Glioblastoma\output_reseg-0017313__N-42-20-T__20240213__135523")

In [4]:
jsonfile = baysor_folder / "segmentation_polygons.json"
d = read_json(jsonfile)
xd = XeniumData(xdfile)

In [5]:
xd.load_all()

Reading annotations...
No `annotations` modality found.
Reading cells...
Reading images...
Reading regions...
No `regions` modality found.
Reading transcripts...


In [6]:
xd.images.nuclei

dask.array<from-zarr, shape=(20539, 28483), dtype=uint16, chunksize=(1024, 1024), chunktype=numpy.ndarray>

In [7]:
xd.show()

Key 'ABCD4' already in layer list.
Key 'ABCD4' already in layer list.
Key 'ABCD4' already in layer list.
Key 'transcript_counts' already in layer list.
Key 'ABO' already in layer list.
Key 'ABO' already in layer list.


In [8]:
xdcrop = xd.crop("Shapes")

In [9]:
xdcrop.show()

In [10]:
savefull = "tmp/full"
savecrop = "tmp/crop"

In [11]:
xd.save(savefull, overwrite=True)

c:\Users\ge37voy\AppData\Local\miniconda3\envs\insitupy\lib\zipfile.py:1514: UserWarning: Duplicate name: '.zattrs'
  return self._open_to_write(zinfo, force_zip64=force_zip64)


In [12]:
xdcrop.save(savecrop, overwrite=True)

c:\Users\ge37voy\AppData\Local\miniconda3\envs\insitupy\lib\zipfile.py:1514: UserWarning: Duplicate name: '.zattrs'
  return self._open_to_write(zinfo, force_zip64=force_zip64)


In [14]:
cropreload = XeniumData(savecrop)
fullreload = XeniumData(savefull)

In [15]:
cropreload.load_all()
fullreload.load_all()

Reading annotations...
No `annotations` modality found.
Reading cells...
Reading images...
Reading regions...
No `regions` modality found.
Reading transcripts...
Reading annotations...
No `annotations` modality found.
Reading cells...


c:\Users\ge37voy\AppData\Local\miniconda3\envs\insitupy\lib\site-packages\anndata\__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
c:\Users\ge37voy\AppData\Local\miniconda3\envs\insitupy\lib\site-packages\anndata\__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


Reading images...
Reading regions...
No `regions` modality found.
Reading transcripts...


In [18]:
cropreload.show()

In [19]:
cropcrop = cropreload.crop("Shapes")

In [22]:
cropcrop.show()

In [23]:
cropcrop.save("tmp/cropcrop")

c:\Users\ge37voy\AppData\Local\miniconda3\envs\insitupy\lib\zipfile.py:1514: UserWarning: Duplicate name: '.zattrs'
  return self._open_to_write(zinfo, force_zip64=force_zip64)


In [24]:
ccreload = XeniumData("tmp/cropcrop")

In [26]:
ccreload.load_all()

Reading annotations...
No `annotations` modality found.
Reading cells...
Reading images...
Reading regions...
No `regions` modality found.
Reading transcripts...


c:\Users\ge37voy\AppData\Local\miniconda3\envs\insitupy\lib\site-packages\anndata\__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [27]:
ccreload.show()

In [17]:
fullreload

XeniumData
Slide ID:	0017313
Sample ID:	N-42-20-T
Data path:	tmp
Data folder:	full
Metadata file:	.xeniumdata
    ➤ images
       nuclei:	(20539, 28483)
    ➤ cells
       matrix
           AnnData object with n_obs × n_vars = 58264 × 314
           obs: 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area'
           var: 'gene_ids', 'feature_types', 'genome'
           obsm: 'spatial'
           varm: 'binned_expression'
       boundaries
           BoundariesData object with 2 entries:
               nuclear
               cellular
    ➤ transcripts
       DataFrame with shape 13317980 x 9

In [13]:
cropreload.show()

In [64]:
cropreload.images.nuclei

dask.array<from-zarr, shape=(2816, 1777), dtype=uint16, chunksize=(2816, 1777), chunktype=numpy.ndarray>

In [70]:
xd.cells.matrix.obs_names.values

array(['aaabfmpc-1', 'aaacfhhg-1', 'aaacngll-1', ..., 'oijdlohl-1',
       'oijdmbco-1', 'oijdmogd-1'], dtype=object)

In [72]:
cropreload.show()

In [35]:
import zarr
import dask.array as da

In [55]:
path = r"C:\Users\ge37voy\Github\InSituPy\notebooks\dev\tmp\crop\images\morphology_mip.zarr.zip"
with zarr.ZipStore(path, mode="r") as zipstore:
    # open store
    img = da.from_zarr(zipstore).persist()
    print(int(img.max()))

5911


In [56]:
print(int(img.max()))

5911


In [57]:
zipstore = zarr.ZipStore(path, mode="r")
img = da.from_zarr(zipstore)

In [49]:
int(img.max())

5911

In [52]:
img = img.persist()

In [53]:
img

dask.array<from-zarr, shape=(2816, 1777), dtype=uint16, chunksize=(2816, 1777), chunktype=numpy.ndarray>

In [50]:
img.compute()

array([[ 34,  24,  26, ...,  50,  41,  33],
       [ 27,  29,  23, ...,  35,  32,  36],
       [ 27,  31,  24, ...,  31,  35,  41],
       ...,
       [654, 706, 720, ..., 310, 300, 287],
       [647, 686, 704, ..., 305, 313, 331],
       [667, 673, 712, ..., 340, 307, 338]], dtype=uint16)

In [41]:
img.shape

(2816, 1777)

ValueError: Attempt to use ZIP archive that was already closed

In [ ]:
xdcrop

In [62]:
list(2)

TypeError: 'int' object is not iterable

In [5]:
xd.load_all()

Reading annotations...
No `annotations` modality found.
Reading cells...
Reading images...
Reading regions...
No `regions` modality found.
Reading transcripts...


In [6]:
xd.parse_baysor(baysor_output=baysor_folder)

Parsing count matrix...
Reading segmentation masks
	Read polygons


c:\Users\ge37voy\AppData\Local\miniconda3\envs\insitupy\lib\site-packages\anndata\_core\anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
c:\Users\ge37voy\AppData\Local\miniconda3\envs\insitupy\lib\site-packages\anndata\_core\anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


	Convert polygons to segmentation mask
Parsing transcripts data...
	Read data
	Merge with existing data


In [7]:
img = xd.images.nuclei[0]

In [8]:
import zarr
import dask.array as da

In [9]:
unzipped = "tmp/unzipped.zarr"
img.to_zarr(unzipped)

In [10]:
store = zarr.open(unzipped, mode="a")
store.attrs["blubb"] = 234

In [11]:
reload = da.from_zarr(unzipped)

In [12]:
reload

dask.array<from-zarr, shape=(20539, 28483), dtype=uint16, chunksize=(1024, 1024), chunktype=numpy.ndarray>

In [13]:
img

dask.array<from-zarr, shape=(20539, 28483), dtype=uint16, chunksize=(1024, 1024), chunktype=numpy.ndarray>

### Test writing of attributes into `zarr.zip`

In [14]:
zipped = "tmp/zipped.zarr.zip"
with zarr.ZipStore(zipped, mode="w") as zipstore:
    img.to_zarr(zipstore)

In [ ]:
# open zipstore
zipstore =  zarr.ZipStore(zipped, mode="a")

# open store
store = zarr.open(zipstore)

# add attributes
store.attrs["blubbzip"] = 3434

# close zip store
zipstore.close()

### Test reading of attributes from `zarr.zip`

In [53]:
zipstore =  zarr.ZipStore(zipped, mode="r")

# open store
store = zarr.open(zipstore)

In [54]:
da.from_zarr(zipstore)

dask.array<from-zarr, shape=(20539, 28483), dtype=uint16, chunksize=(1024, 1024), chunktype=numpy.ndarray>

In [50]:
store

<zarr.core.Array (20539, 28483) uint16>

In [51]:
store.attrs.asdict()

{'blubbzip': 3434}

In [25]:
zipstore.close()

In [26]:
store.attrs.asdict()

{'blubbzip': 3434}

In [30]:
store.attrs

In [33]:
savepath = Path("tmp/savetest")

In [31]:
xd.save(savepath, overwrite=True)

c:\Users\ge37voy\AppData\Local\miniconda3\envs\insitupy\lib\zipfile.py:1514: UserWarning: Duplicate name: '.zattrs'
  return self._open_to_write(zinfo, force_zip64=force_zip64)


In [58]:
xd.images.metadata['nuclei']

{'file': 'morphology_mip.ome.tif',
 'shape': (20539, 28483),
 'subresolutions': 6,
 'axes': 'YX',
 'OME': {'xmlns': 'http://www.openmicroscopy.org/Schemas/OME/2016-06',
  'xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
  'Creator': 'tifffile.py 2022.10.10',
  'UUID': 'urn:uuid:82c8f6e2-b198-11ee-96eb-3cecefcca217',
  'xsi:schemaLocation': 'http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd',
  'Plate': {'ID': 'Plate:1',
   'WellOriginX': '-0.0',
   'WellOriginXUnit': 'µm',
   'WellOriginY': '-0.0',
   'WellOriginYUnit': 'µm'},
  'Instrument': {'ID': 'Instrument:1',
   'Microscope': {'Manufacturer': '10x Genomics', 'Model': 'Xenium'}},
  'Image': {'ID': 'Image:0',
   'Name': 'Image0',
   'InstrumentRef': {'ID': 'Instrument:1'},
   'Pixels': {'DimensionOrder': 'XYZCT',
    'ID': 'Pixels:0',
    'SizeC': '1',
    'SizeT': '1',
    'SizeX': '28483',
    'SizeY': '20539',
    'SizeZ': '1',
    'Type': 'uint16',
    'Physical

In [36]:
imgpath = savepath / "images/morphology_mip.zarr.zip"

In [39]:
reloaded = da.from_zarr(imgpath)

In [45]:
with zarr.ZipStore(imgpath, mode="r") as zipstore:
    # open store
    store = zarr.open(zipstore)
    print(store.attrs.asdict())

{'OME': {'Creator': 'tifffile.py 2022.10.10', 'Image': {'ID': 'Image:0', 'InstrumentRef': {'ID': 'Instrument:1'}, 'Name': 'Image0', 'Pixels': {'Channel': {'ID': 'Channel:0:0', 'Name': 'DAPI', 'SamplesPerPixel': '1'}, 'DimensionOrder': 'XYZCT', 'ID': 'Pixels:0', 'PhysicalSizeX': '0.2125', 'PhysicalSizeY': '0.2125', 'SizeC': '1', 'SizeT': '1', 'SizeX': '28483', 'SizeY': '20539', 'SizeZ': '1', 'TiffData': {'PlaneCount': '1'}, 'Type': 'uint16'}}, 'Instrument': {'ID': 'Instrument:1', 'Microscope': {'Manufacturer': '10x Genomics', 'Model': 'Xenium'}}, 'Plate': {'ID': 'Plate:1', 'WellOriginX': '-0.0', 'WellOriginXUnit': 'µm', 'WellOriginY': '-0.0', 'WellOriginYUnit': 'µm'}, 'UUID': 'urn:uuid:82c8f6e2-b198-11ee-96eb-3cecefcca217', 'xmlns': 'http://www.openmicroscopy.org/Schemas/OME/2016-06', 'xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance', 'xsi:schemaLocation': 'http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd'}, 'axes': 'YX',

In [48]:
imgpath.name.split(".", maxsplit=1)[-1]

'zarr.zip'

In [42]:
reloaded.attrs

AttributeError: 'Array' object has no attribute 'attrs'

In [ ]:
with zarr.DirectoryStore(unzipped) as store:
    store.attrs["blubb"] = 234

AttributeError: 'DirectoryStore' object has no attribute 'attrs'

In [ ]:
store = zarr.ZipStore("tmp/testtest.zarr.zip", mode="w")

In [ ]:
g = zarr.group(store)

In [ ]:
g.attrs["check"] = 42

In [ ]:
store.close()

In [ ]:
read = zarr.ZipStore("tmp/testtest.zarr.zip", mode="r")

In [ ]:
zarr.open(read).attrs["check"]

42

# As shown above I was able to set the attributes in a ZipStore!!!

In [ ]:
store = zarr.group()

In [ ]:
store

<zarr.hierarchy.Group '/'>

In [ ]:
store.attrs

In [ ]:
store.group()

AttributeError: 'ZipStore' object has no attribute 'group'

In [ ]:
attrs = zarr.attrs.Attributes(store=store)

In [ ]:
attrs["blubb"] = 234

In [ ]:
store.close()

In [ ]:
store = zarr.ZipStore("tmp/testtest.zarr.zip", mode="r")

In [ ]:
with zarr.ZipStore("tmp/testtest.zarr.zip", mode="w") as store:
    with zarr.open(store=store) as s:
        for k,v in xd.images.metadata.items():
            s.attrs[k] = v

    img.to_zarr(store)

ContainsGroupError: path '' contains a group

In [ ]:
with zarr.DirectoryStore("tmp/testtest.zarr") as store:
    for k,v in xd.images.metadata.items():
        store.attrs[k] = v
    img.to_zarr(store)

AttributeError: 'DirectoryStore' object has no attribute 'attrs'

In [ ]:
store = zarr.open("tmp/testtest.zarr", mode="w")

In [ ]:
for k,v in xd.images.metadata.items():
    store.attrs[k] = v

In [ ]:
type(store)

zarr.hierarchy.Group

In [ ]:
store.attrs["blubb"] = 42

In [ ]:
img.to_zarr("tmp/testimg.zarr")

In [ ]:
import dask.array as da

In [ ]:
savepath = "tmp/savetest"

In [ ]:
xd.save(savepath, overwrite=True)

NotImplementedError: 